# Latent_Dirichlet_Allocation

In [ ]:
import os
import sys

parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)

from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from sklearn.model_selection import train_test_split
from gensim.models import LdaModel
from Preprocess import preprocess
from Plot import plot_df

# Run the preprocessing 
df = preprocess('Data/articles.json')

from sklearn.model_selection import train_test_split
from gensim import corpora


# Creating the dictionary for the training corpus
dictionary = corpora.Dictionary(df['content'])
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Converting the training corpus to bag-of-words representation
corpus = [dictionary.doc2bow(doc) for doc in df['content']]

# Hyperparameter_Tuning

In [7]:
# Set up the parameter grid for tuning
param_grid = {
    'num_topics': [5, 10, 20, 28],  # Test different values for the number of topics
    'alpha': [0.01, 0.1, 1.0],   # Test different values for alpha
    'eta': [0.01, 0.1, 1.0]       # Test different values for beta (eta in Gensim)
}

# Perform grid search
best_coherence = -1
best_model = None

for num_topics in param_grid['num_topics']:
    for alpha in param_grid['alpha']:
        for eta in param_grid['eta']:
            # Train the LDA model
            model = LdaModel(
                corpus=corpus,
                id2word=dictionary,
                num_topics=num_topics,
                alpha=alpha,
                eta=eta,
                random_state=42
            )
            
            # Compute coherence score
            coherence_model = gensim.models.CoherenceModel(model=model, texts= df['content'], dictionary=dictionary, coherence='c_v')
            coherence = coherence_model.get_coherence()
            
            # Check if this model has the highest coherence so far
            if coherence > best_coherence:
                best_coherence = coherence
                best_model = model

# Get the best parameters
best_num_topics = best_model.num_topics
best_alpha = best_model.alpha
best_eta = best_model.eta


print("Best number of topics:", best_num_topics)
print("Best alpha value:", best_alpha[0])
print("Best eta value:", best_eta[0])

Best number of topics: 28
Best alpha value: 0.01
Best eta value: 0.0001


# Train_Model

In [4]:
from gensim.models import LdaModel

# Set the best parameters
num_topics = 28
best_alpha = 0.01
best_eta = 0.0001

# Train the LDA model with the best hyperparameters
model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
)

# Compute the perplexity on the test corpus
perplexity = model.log_perplexity(corpus)

# Print the perplexity value
print("Perplexity:", perplexity)

# Save the trained model
model_path = "lda_model"
model.save(model_path)

Perplexity: -7.817743339406679


# Coherence & Topics

In [6]:
from Plot import print_coherence
from gensim import corpora
from Preprocess import preprocess

df = preprocess('Data/articles.json')


dictionary = corpora.Dictionary(df['content'])

dic = dictionary
print_coherence(dic, topic_words,df['content'])

Topic: health,peopl,award,commun,youth,hospit,young,	Coherence: 0.54
Topic: space,peopl,world,plastic,secur,commonwealth,	Coherence: 0.47
Topic: anim,woman,welfar,wast,pet,ban,protect,puppi,	Coherence: 0.40
Topic: fish,fisheri,river,angler,protect,angl,stock,	Coherence: 0.71
Topic: payment,farmer,scheme,manag,claim,grant,engla	Coherence: 0.73
Topic: develop,climat,fund,trade,intern,busi,global,	Coherence: 0.60
Topic: busi,food,industri,secretari,research,farm,pr	Coherence: 0.58
Topic: wast,permit,environment,oper,compani,offic,bu	Coherence: 0.57
Topic: flood,protect,commun,marin,risk,coastal,peopl	Coherence: 0.60
Topic: project,council,qualiti,plant,environment,fun	Coherence: 0.46
Topic: water,compani,bath,qualiti,pollut,river,engla	Coherence: 0.64
Topic: fish,wast,illeg,offic,crime,enforc,fisheri,po	Coherence: 0.72
Topic: peopl,water,mine,natur,coal,england,health,me	Coherence: 0.43
Topic: natur,england,speci,protect,wildlif,habitat,b	Coherence: 0.70
Topic: project,train,british,defen

# Dynamic LDA

In [ ]:
import os
df_copy = df.copy()

# Calculate the topic proportions for each document
topic_proportions = model.get_document_topics(corpus, minimum_probability=0.0)

# Add the topic proportions to the DataFrame
for i, topic in enumerate(topic_words):
    topic_name = str(','.join(topic[:4]))
    df_copy[topic_name] = [doc_topic[i][1] for doc_topic in topic_proportions]
    
topic_means = df_copy.groupby('year-month').mean()

plot_df(topic_means, 'LDA', span_size=7, add_error_bars=True)
